In [1]:
# This notebook runs from the projects root directory.
import os
PROJECT_ROOT_DIRECTORY = 'senior_project'
while os.path.basename(os.getcwd()) != PROJECT_ROOT_DIRECTORY:
    os.chdir(os.pardir)
print('Current working directory:', os.getcwd())

Current working directory: c:\Users\pavan\Desktop\senior_project


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import time
from stable_baselines3 import DQN, A2C, PPO, DDPG
from environments.continuous_env import PortfolioManagementEnv
from stable_baselines3.common.noise import NormalActionNoise

In [3]:
experiment_id = f'EXPERIMENT_9_{int(time.time())}'
print(experiment_id)

EXPERIMENT_9_1678768311


In [4]:
models_dir = f'experiments/models/{experiment_id}'
log_dir = f'experiments/logs'

if not os.path.exists(models_dir):
    os.makedirs(models_dir)

if not os.path.exists(log_dir):
    os.makedirs(log_dir)

In [5]:
train_df = pd.read_csv('data\dow_10_stationary_features_train.csv', index_col=0, parse_dates=True)

TICKERS = ['AXP', 'AAPL', 'BA', 'GS', 'INTC', 'JNJ', 'KO', 'NKE', 'PG', 'DIS']
FEATURES = ['RETURN_12', 'RETURN_26', 'RETURN_60', 'STD_20']

RETURN_COLS = ['RISK_FREE'] + [ticker for ticker in TICKERS]
FEATURE_COLS = RETURN_COLS + [f'{ticker}_{feature}' for ticker in TICKERS for feature in FEATURES]

In [6]:
train_env = PortfolioManagementEnv(
    train_df, 
    RETURN_COLS, 
    FEATURE_COLS, 
    window_size = 30,
    episode_length = 225,
    drawdown_penalty_weight = 1,
    allocations_in_obs = True,
)

In [7]:
n_actions = train_env.action_space.shape[-1]
action_noise = NormalActionNoise(mean=np.zeros(n_actions), sigma=0.1 * np.ones(n_actions))

model = DDPG(
    'MlpPolicy',
    train_env,
    verbose=1,
    tensorboard_log=log_dir,
    seed=5,
    learning_rate=0.001,
    action_noise=action_noise,
    buffer_size=100_000,
)

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [8]:
TIMESTEPS = 10_000 # number of timesteps between saves
for i in tqdm(range(1, 300)):
    model.learn(total_timesteps=TIMESTEPS, reset_num_timesteps=False, tb_log_name=experiment_id)
    model.save(f'{models_dir}/{TIMESTEPS*i}')

  0%|          | 0/299 [00:00<?, ?it/s]

Logging to experiments/logs\EXPERIMENT_9_1678768311_0
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 225      |
|    ep_rew_mean     | 4.89     |
| time/              |          |
|    episodes        | 4        |
|    fps             | 93       |
|    time_elapsed    | 9        |
|    total_timesteps | 900      |
| train/             |          |
|    actor_loss      | -0.254   |
|    critic_loss     | 0.000203 |
|    learning_rate   | 0.001    |
|    n_updates       | 675      |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 225      |
|    ep_rew_mean     | 7.94     |
| time/              |          |
|    episodes        | 8        |
|    fps             | 93       |
|    time_elapsed    | 19       |
|    total_timesteps | 1800     |
| train/             |          |
|    actor_loss      | -0.701   |
|    critic_loss     | 0.000449 |
|    learning_rate   | 0.001

KeyboardInterrupt: 